In [1]:
print 'load libraries...'

load libraries...


In [2]:
# Aim: 

In [3]:
# Add path in order to load utility functions
import sys
sys.path.append(r"N:\18. Hot Spot\Custom Modules\Utils")
sys.path.append(r"N:\18. Hot Spot\Custom Modules\ETLMonthly")

In [4]:
# setup environment and import libraries
import os
import re
import datetime
import time
import getpass
import pandas as pd
import numpy as np
from datetime import datetime, date
from datetime import timedelta
import calendar as cd
from collections import OrderedDict
path=r'C:\ProgramData\Anaconda2\instantclient_12_2'
os.environ['PATH']=path
import cx_Oracle
import file_utils
import db_utils
import config
from sqlalchemy import create_engine
#set working directory
os.chdir(config.working_dir)

In [5]:
# db credentials authentication

In [6]:
print 'connect to database...'
conn_string = db_utils.db_connect(config.db_username,config.db_pwd)

connect to database...
Testing Connection...
Connection Succesful!


In [7]:
mtch = re.match('.*@', conn_string)
un, pwd = mtch.group().rstrip('@').split('/')

In [8]:
con = cx_Oracle.connect(conn_string)
cs_retail_loan_cust_data = pd.read_sql_query('select bra_code, cus_num from cs_retail_loan_cust', con)
con.close()

In [9]:
# get airtime recharge related to only the retail_loan guys

for i, f in enumerate(file_utils.get_files(r"N:\10. Executive\Original\ETL\airtime_recharges")):
    print i+1, '. ', f

    cur_cs_airtime_df = file_utils.unzip(f).merge(cs_retail_loan_cust_data[['BRA_CODE','CUS_NUM']],
                                                  how='inner', on=['BRA_CODE','CUS_NUM'])
    
    if_exists_var = 'append'
    if i==0:
        if_exists_var = 'replace'
    
    cur_cs_airtime_df['REFERENCE_DATE']=pd.to_datetime(cur_cs_airtime_df['REFERENCE_DATE'], format="%d%b%Y")
    cur_cs_airtime_df['CTRL_DATE']=pd.to_datetime(cur_cs_airtime_df['CTRL_DATE'], format="%d%b%Y")
    
    conn = create_engine('oracle+cx_oracle://{}:{}@dwdrscan.gtbank.com:1521/?service_name=odidb'.format(un, pwd))    
    cur_cs_airtime_df.to_sql('cs_airtime', conn, index=False, if_exists=if_exists_var, chunksize=10000,
                              dtype=db_utils.sqlcol(cur_cs_airtime_df))
    

1 .  N:\10. Executive\Original\ETL\airtime_recharges\05Mar2020.csv.gz
2 .  N:\10. Executive\Original\ETL\airtime_recharges\28Feb2018.csv.gz
3 .  N:\10. Executive\Original\ETL\airtime_recharges\28Feb2019.csv.gz
4 .  N:\10. Executive\Original\ETL\airtime_recharges\29Feb2020.csv.gz
5 .  N:\10. Executive\Original\ETL\airtime_recharges\30Apr2018.csv.gz
6 .  N:\10. Executive\Original\ETL\airtime_recharges\30Apr2019.csv.gz
7 .  N:\10. Executive\Original\ETL\airtime_recharges\30Jun2018.csv.gz
8 .  N:\10. Executive\Original\ETL\airtime_recharges\30Jun2019.csv.gz
9 .  N:\10. Executive\Original\ETL\airtime_recharges\30Nov2018.csv.gz
10 .  N:\10. Executive\Original\ETL\airtime_recharges\30Nov2019.csv.gz
11 .  N:\10. Executive\Original\ETL\airtime_recharges\30Sep2018.csv.gz
12 .  N:\10. Executive\Original\ETL\airtime_recharges\30Sep2019.csv.gz
13 .  N:\10. Executive\Original\ETL\airtime_recharges\31Aug2018.csv.gz
14 .  N:\10. Executive\Original\ETL\airtime_recharges\31Aug2019.csv.gz
15 .  N:\10. Ex

In [10]:
con = cx_Oracle.connect(conn_string)
cs_airtime_info_df = pd.read_sql_query('''
select a.bra_code, a.cus_num, count(1) airtime_nbr_month_active,
nvl(avg(case when total_airtime_count>self_airtime_count then 1 end),0) airtime_do_third_party,
round(sum(total_airtime_count)/12) airtime_avg_per_month_count
from cs_retail_loan_cust a, cs_airtime b
where a.bra_code=b.bra_code and a.cus_num=b.cus_num
and reference_date between trunc(add_months(a.date_open, -12),'month') and trunc(add_months(a.date_open, -1),'month')
group by a.bra_code, a.cus_num
''', con)

print len(cs_airtime_info_df)

142199


In [11]:
cs_retail_loan_cust_data.merge(cs_airtime_info_df, on=['BRA_CODE','CUS_NUM'],
                             how='left').fillna(0).to_csv('cs_airtime_info.csv', index=False)